## Type of Joins in Hive
<ul>
    <li>Inner join in Hive</li>
    <li>Left Outer Join in Hive</li>
    <li>Right Outer Join in Hive</li>
    <li>Full Outer Join in Hive</li>
</ul>

Melwin its similar to the MYSQL JOINS

### Map Side Joins
<ul>
    <li>Map side join is a process where joins between two tables are performed in the Map
phase without the involvement of Reduce phase</li>
    <li>Map-side Joins allows a table to get loaded into memory ensuring a very fast join
operation, performed entirely within a mapper and that too without having to use
both map and reduce phases.</li>
</ul>

<p>Map side join is usually <b>used when one data set is large and the other data set is small.</b></p>

<pre>
    hive.auto.convert.join = true 
</pre>
<pre>
    CREATE TABLE IF NOT EXISTS dataset1 
    ( 
    id int, 
    first_name String, 
    last_name String, 
    email String, 
    gender String, 
    ip_address String
    ) 
    row format delimited fields terminated BY ','
    tblproperties("skip.header.line.count"="1"); 
    
    CREATE TABLE IF NOT EXISTS dataset2 
    ( 
    id int, 
    first_name String, 
    last_name String
    ) 
    row format delimited fields terminated BY ',' 
    tblproperties("skip.header.line.count"="1"); 
    
    load data local inpath '/home/saif/LFS/dataset1.csv' into table dataset1; 
    load data local inpath '/home/saif/LFS/dataset2.csv' into table dataset2;
    
    SELECT /*+ MAPJOIN(dataset2) */ dataset1.first_name, dataset1.id, dataset2.id FROM
    dataset1 JOIN dataset2 ON dataset1.first_name = dataset2.first_name; 
</pre>

### Bucket-Map Join


Bucket map join is used when the joining tables are large and are <b>bucketed on the join column</b>.

<pre>
set hive.optimize.bucketmapjoin = true 
<pre>

<pre>
CREATE TABLE IF NOT EXISTS dataset1_bucketed 
( 
id int,
first_name String, 
last_name String, 
email String, 
gender String, 
ip_address String) 
clustered by (first_name) into 4 buckets 
row format delimited fields terminated BY ',';

CREATE TABLE IF NOT EXISTS dataset2_bucketed 
(
id int,
first_name String, 
last_name String) 
clustered by(first_name) into 8 buckets 
row format delimited fields terminated BY ',' ;

insert into dataset1_bucketed select * from dataset1;
insert into dataset2_bucketed select * from dataset2;

SELECT /*+ MAPJOIN(dataset2_bucketed) */ dataset1_bucketed.first_name, dataset1_bucketed.id, dataset2_bucketed.id FROM dataset1_bucketed JOIN dataset2_bucketed ON dataset1_bucketed.first_name = dataset2_bucketed.first_name;
</pre>


### Sort Merge Bucket(SMB) Map Join

<p><b>Sort Merge Bucket (SMB)</b> join in hive is mainly used as there is no limit on file or partition or table join. <br> 
   SMB join can best be used when the tables are large. <br>
   In SMB join the columns are bucketed and sorted using the join columns. All tables should have the same number of buckets in SMB join.</p>

<pre>
set hive.input.format=org.apache.hadoop.hive.ql.io.BucketizedHiveInputFormat;
set hive.optimize.bucketmapjoin = true;
set hive.optimize.bucketmapjoin.sortedmerge = true; 
</pre>

<pre>
CREATE TABLE IF NOT EXISTS dataset2_bucketed1 (
id int,
first_name String, 
last_name String) 
clustered by(first_name) into 4 buckets 
row format delimited fields terminated BY ',' ;


insert overwrite table dataset2_bucketed1 select * from dataset2 sort by first_name;


SELECT /*+ MAPJOIN(dataset2_bucketed1) */dataset1_bucketed.first_name, dataset1_bucketed.id, dataset2_bucketed1.id FROM dataset1_bucketed JOIN dataset2_bucketed1 ON dataset1_bucketed.first_name = dataset2_bucketed1.first_name ;

</pre>